# Assignment 3

Import libraries and define common helper functions

In [1]:
import os
import sys
import gzip
import json
from pathlib import Path
import csv

import pandas as pd
import s3fs
import pyarrow as pa
from pyarrow.json import read_json
import pyarrow.parquet as pq
import fastavro
import pygeohash
import snappy
import jsonschema
from jsonschema.exceptions import ValidationError


path='E:\Documents\GitHub\dsc650\dsc650\assignments\assignment03'

current_dir = Path(os.getcwd()).absolute()
schema_dir = current_dir.joinpath('schemas')
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)


def read_jsonl_data():
    ## assignment 3 (to read the data from the repo instead of S3)
    src_data_path = '../../../data/processed/openflights/routes.jsonl.gz'
    with gzip.open(src_data_path, 'rb') as f:
        records = [json.loads(line) for line in f.readlines()]
    return records

Load the records from https://storage.budsc.midwest-datascience.com/data/processed/openflights/routes.jsonl.gz 

In [2]:
records = read_jsonl_data()

## 3.1

### 3.1.a JSON Schema

In [6]:
def validate_jsonl_data(records):
    schema_path = schema_dir.joinpath('routes-schema.json')
    validation_csv_path = results_dir.joinpath('validation-results.csv')
    with open(schema_path) as f:
        schema = json.load(f)
        
    with open(validation_csv_path, 'w') as f:    
        for i, record in enumerate(records):
            try:
                ## Validate record 
                jsonschema.validate(instance = records, schema=schema)
                
            except ValidationError as e:
                ## Print message if invalid record
                print("Invalid record. Please try again")
                

validate_jsonl_data(records)

### 3.1.b Avro

In [7]:
def create_avro_dataset(records):
    schema_path = schema_dir.joinpath('routes.avsc')
    data_path = results_dir.joinpath('routes.avro')
    ## TODO: Use fastavro to create Avro dataset
    with open(schema_path, "rb") as schema_file:
        schema = fastavro.schema.load_schema(schema_file.name)
    with open(data_path, 'wb') as out:
        fastavro.writer(out, schema, records)
        
create_avro_dataset(records)

### 3.1.c Parquet

In [8]:
records2 = pd.DataFrame(records)

In [13]:
def create_parquet_dataset():
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    ## assignment 3 (to read the data from the repo instead of S3)
    src_data_path = '../../../data/processed/openflights/routes.jsonl.gz'
    with gzip.open(src_data_path, 'rb') as f:
            ## TODO: Use Apache Arrow to create Parquet table and save the dataset
            records2 = pd.DataFrame([json.loads(line) for line in f.readlines()])
            pa_table = pa.table(records2)
            pq.write_table(pa_table, results_dir.joinpath("routes.parquet"))
create_parquet_dataset()

### 3.1.d Protocol Buffers

In [3]:
sys.path.insert(0, os.path.abspath('routes_pb2'))

import routes_pb2

def _airport_to_proto_obj(airport):
    obj = routes_pb2.Airport()
    if airport is None:
        return None
    if airport.get('airport_id') is None:
        return None

    obj.airport_id = airport.get('airport_id')
    if airport.get('name'):
        obj.name = airport.get('name')
    if airport.get('city'):
        obj.city = airport.get('city')
    if airport.get('iata'):
        obj.iata = airport.get('iata')
    if airport.get('icao'):
        obj.icao = airport.get('icao')
    if airport.get('altitude'):
        obj.altitude = airport.get('altitude')
    if airport.get('timezone'):
        obj.timezone = airport.get('timezone')
    if airport.get('dst'):
        obj.dst = airport.get('dst')
    if airport.get('tz_id'):
        obj.tz_id = airport.get('tz_id')
    if airport.get('type'):
        obj.type = airport.get('type')
    if airport.get('source'):
        obj.source = airport.get('source')

    obj.latitude = airport.get('latitude')
    obj.longitude = airport.get('longitude')

    return obj


def _airline_to_proto_obj(airline):
    obj = routes_pb2.Airline()
    if not airline.get('name'):
        return None
    if not airline.get('airline_id'):
        return None
    obj.airline_id = airline.get('airline_id')
    obj.name = airline.get('name')
    if airline.get('alias'):
        obj.alias = airline.get('alias')
    if airline.get('iata'):
        obj.iata = airline.get('iata')
    if airline.get('icao'):
        obj.icao = airline.get('icao')
    if airline.get('callsign'):
        obj.callsign = airline.get('callsign')
    if airline.get('country'):
        obj.country = airline.get('country')
    if airline.get('active'):
        obj.active = airline.get('active')
    else:
        obj.active = False
        
    ## TODO: Create an Airline obj using Protocol Buffers API
    return obj


def create_protobuf_dataset(records):
    routes = routes_pb2.Routes()
    for record in records:
        route = routes_pb2.Route()
        airline = _airline_to_proto_obj(record.get('airline', {}))
        if airline:
            route.airline.CopyFrom(airline)
        src_airport = _airport_to_proto_obj(record.get('src_airport', {}))
        if src_airport:
            route.src_airport.CopyFrom(src_airport)
        dst_airport = _airport_to_proto_obj(record.get('dst_airport', {}))
        if dst_airport:
            route.dst_airport.CopyFrom(dst_airport)
        if 'codeshare' in record and record['codeshare'] is not None:
            route.codeshare = record['codeshare']
        else:
            route.codeshare = False
            
        routes.route.append(route)

    data_path = results_dir.joinpath('routes.pb')

    with open(data_path, 'wb') as f:
        f.write(routes.SerializeToString())
        
    compressed_path = results_dir.joinpath('routes.pb.snappy')
    
    with open(compressed_path, 'wb') as f:
        f.write(snappy.compress(routes.SerializeToString()))
        
create_protobuf_dataset(records)

## 3.1.e Comparison

In [5]:
import os
import csv

 

def get_file_size(file_path):
    """Get the size of a file in bytes"""
    return os.stat(file_path).st_size

 

# File paths
avro_file = r"E:\Documents\GitHub\dsc650\dsc650\assignments\assignment03\results\routes.avro"
pb_file = r"E:\Documents\GitHub\dsc650\dsc650\assignments\assignment03\results\routes.pb"
pb_snappy_file = r"E:\Documents\GitHub\dsc650\dsc650\assignments\assignment03\results\routes.pb.snappy"
output_file = r"E:\Documents\GitHub\dsc650\dsc650\assignments\assignment03\results\comparison.csv"

 

# Get file sizes
avro_size = get_file_size(avro_file)
pb_size = get_file_size(pb_file)
pb_snappy_size = get_file_size(pb_snappy_file)

 

# Write results to CSV file
with open(output_file, mode='w', newline='') as csv_file:
    fieldnames = ['File Format', 'Uncompressed Size (bytes)', 'Compressed Size (bytes)']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({'File Format': 'Avro', 'Uncompressed Size (bytes)': avro_size, 'Compressed Size (bytes)': 'N/A'})
    writer.writerow({'File Format': 'Protocol Buffers', 'Uncompressed Size (bytes)': pb_size, 'Compressed Size (bytes)': 'N/A'})
    writer.writerow({'File Format': 'Protocol Buffers (with Snappy compression)', 'Uncompressed Size (bytes)': pb_size, 'Compressed Size (bytes)': pb_snappy_size})
    
print("Comparison results saved to:", output_file)

Comparison results saved to: E:\Documents\GitHub\dsc650\dsc650\assignments\assignment03\results\comparison.csv


## 3.2

### 3.2.a Simple Geohash Index

In [17]:
def create_hash_dirs(records):
    geoindex_dir = results_dir.joinpath('geoindex')
    geoindex_dir.mkdir(exist_ok=True, parents=True)
    hashes = []
    for record in records:
        src_airport = record.get('src_airport', {})
        if src_airport:
            latitude = src_airport.get('latitude')
            longitude = src_airport.get('longitude')
            if latitude and longitude:
                encoded= pygeohash.encode(latitude, longitude)
                src_airport['geohash']=encoded
                hashes.append(encoded)
                ## TODO: use pygeohash.encode() to assign geohashes to the records and complete the hashes list
    hashes.sort()
    three_letter = sorted(list(set([entry[:3] for entry in hashes])))
    hash_index = {value: [] for value in three_letter}
    for record in records:
        geohash = record.get('geohash')
        if geohash:
            hash_index[geohash[:3]].append(record)
    for key, values in hash_index.items():
        output_dir = geoindex_dir.joinpath(str(key[:1])).joinpath(str(key[:2]))
        output_dir.mkdir(exist_ok=True, parents=True)
        output_path = output_dir.joinpath('{}.jsonl.gz'.format(key))
        with gzip.open(output_path, 'w') as f:
            json_output = '\n'.join([json.dumps(value) for value in values])
            f.write(json_output.encode('utf-8'))
create_hash_dirs(records)

### 3.2.b Simple Search Feature

In [24]:
def airport_search(latitude, longitude):
    hash_code = pygeohash.encode(latitude, longitude)
    distances = {}
    for record in records:
        src_airport = record.get('src_airport', {})
        if src_airport:
            geohash = src_airport.get('geohash')
            if geohash:
                dist = pygeohash.geohash_approximate_distance(geohash, hash_code) 
                distances[src_airport.get('name')] = dist
    smallest = min(distances, key=distances.get)
    return smallest
    
airport_search(41.1499988, -95.91779)

'Eppley Airfield'